In [2]:
import xarray as xr
import matplotlib.pyplot as plt
from pyclim_noresm.general_util_funcs import global_avg
import pandas as pd

In [3]:
time_slice = snakemake.params.get('time_slice',slice(None,None))
vmin = snakemake.params.get('vmin',-2.5)
vmax = snakemake.params.get('vmax', 1)

paths = snakemake.input.paths
vname = snakemake.wildcards.vName


In [7]:
dsets = {}
for path in paths:

    ds = xr.open_dataset(path)
    source_id = ds.source_id
    ds = ds[vname].isel(year=time_slice)
    ds = ds.rename({'year':'time'})
    ds = global_avg(ds)
    dsets[source_id] = ds.values
    time=ds.time.values

df=pd.DataFrame(dsets, index=time)

In [5]:
fig, ax = plt.subplots(figsize=(14,6))
ax.axhline(y=0, color='k', linewidth=3.1)
df.plot(ax=ax, alpha = 0.3, color=["#1845fb", "#ff5e02", "#c91f16", 
                                   "#c849a9", "#adad7d", "#86c8dd", "#578dff", "#656364"],
       legend=False)
df.rolling(5,).mean().plot(ax=ax, 
                          color=["#1845fb", "#ff5e02", "#c91f16", "#c849a9", 
                                 "#adad7d", "#86c8dd", "#578dff", "#656364"],
                         linewidth=2.1)
ax.set_ylabel(f'$\Delta ${vname} histSST histSST-piAer \n W m-2')

ax.set_ylim(vmin,vmax)
plt.savefig(snakemake.output.outpath, dpi=144, bbox_inches='tight')